给定一个RPA流程需求描述，先让LLM重新描述一个需求和执行步骤，然后生成伪代码，再根据API文档和伪代码，生成实际的代码。

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA


openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)


embeddings = OpenAIEmbeddings(model='text-embedding-ada-002',
                              deployment='emb0614',
                              openai_api_base=os.getenv("OPENAI_API_BASE"),
                              openai_api_type=os.getenv("OPENAI_API_TYPE"),
                              chunk_size=1)

docsearch = Chroma(persist_directory='apidocs', embedding_function=embeddings)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.3, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
template_task = '''你是个非常优秀的RPA程序员，你的任务是根据RPA流程的需求描述和步骤描述，生成一个伪代码表示的RPA流程。
请一步步思考，确保正确理解实现该流程的所有必要条件，然后更新步骤描述，确保新的步骤描述包含了所有的上下文信息，以方便后面的代码。

流程描述: {description}

任务步骤：{steps}

重新描述实现该RPA流程的具体步骤，包括每个步骤的输入输出和实现功能：'''

prompt_task = PromptTemplate(input_variables=['description', 'steps'], template=template_task)
task_chain = LLMChain(llm=llm, prompt=prompt_task, output_key='revised')

In [4]:
template_draft = '''你是个非常优秀的RPA程序员，请根据RPA流程描述和实现步骤描述，先生成一个用伪代码表示的RPA流程草稿。

流程描述: {description}

详细步骤：{revised}

生成的伪代码RPA流程是：'''

prompt_draft = PromptTemplate(input_variables=['description', 'revised'], template=template_draft)
draft_chain = LLMChain(llm=llm, prompt=prompt_draft, output_key='draft')

In [16]:
template_code = '''你是个非常优秀的RPA程序员，请根据RPA流程描述和流程草稿，实现一个由基本操作组成的伪代码RPA流程。
基本操作是指计算机能够执行的最简单任务，比如打开文件、保存文件、打开浏览器、点击按钮、修改文本等。

流程描述: {description}
流程草稿: {draft}

生成的RPA流程伪代码：'''

prompt_code = PromptTemplate(input_variables=['description', 'draft'], template=template_code)

code_chain = LLMChain(llm=llm, prompt=prompt_code, output_key='code')

In [17]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[task_chain, draft_chain, code_chain],
    input_variables=['description', 'steps'],
    # Here we return multiple variables
    output_variables=["revised", "draft", "code"],
    verbose=True)

In [18]:
task_description = '''通过RPA机器人，进入网银系统获取前一个工作日的资金流水数据，
按数据类型写入到模板表对应的银行列中，并下载前一工作日电子回单，生成文件清单并打印，
再发送资金日报及电子回单清单给对应的业务人员。
'''

task_steps = '''操作步骤及说明
1.	业务人员配置好银行账号数据表，和邮箱账号数据；
2.	读取账号配置表，根据账号信息自动登录网银系统；
3.	获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；
4.	下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；
5.	将日报excel及回、流水文件夹进行压缩；
6.	邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；
7.	业务人员接收邮件文件，进行确认操作。
'''

In [19]:
result = overall_chain({'description': task_description, 'steps': task_steps})



> Entering new SequentialChain chain...

> Finished chain.


In [20]:
print(result['revised'])

1. 读取银行账号数据表和邮箱账号数据表，获取账号信息；
   输入：银行账号数据表、邮箱账号数据表；
   输出：账号信息；
   功能：获取账号信息，为后续自动登录网银系统和发送邮件做准备。

2. 自动登录网银系统；
   输入：账号信息；
   输出：已登录的网银系统；
   功能：自动登录网银系统，为后续获取前一天历史余额数据和下载电子回单、网银流水做准备。

3. 获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；
   输入：已登录的网银系统、资金日报表模板、银行账号信息；
   输出：资金日报表模板对应银行账户余额栏的历史余额数据；
   功能：获取前一天历史余额数据，为后续生成资金日报和发送邮件做准备。

4. 下载前一天电子回单、网银流水，按照固定规则命名，并在日志记录表中写入记录；
   输入：已登录的网银系统、下载路径、命名规则、日志记录表；
   输出：下载的电子回单、网银流水文件；
   功能：下载前一天电子回单、网银流水文件，并按照固定规则命名，同时在日志记录表中写入记录。

5. 将日报excel及回、流水文件夹进行压缩；
   输入：资金日报表模板、电子回单、网银流水文件夹路径；
   输出：压缩后的文件夹；
   功能：将资金日报表模板、电子回单、网银流水文件夹进行压缩，为后续发送邮件做准备。

6. 邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；
   输入：压缩后的文件夹、邮箱账号信息、邮件内容；
   输出：发送成功的邮件；
   功能：发送邮件提醒运行异常，并将压缩后的文件夹发送到指定邮箱。

7. 业务人员接收邮件文件，进行确认操作。
   输入：接收的邮件、确认操作；
   输出：确认结果；
   功能：业务人员接收邮件文件，并进行确认操作。


In [21]:
print(result['draft'])

1. Read bank account data table and email account data table to get account information
2. Automatically log in to online banking system using account information
3. Get historical balance data from the previous business day and write it into the corresponding bank account balance column in the cash flow template
4. Download electronic receipts and online banking transaction records from the previous business day, name them according to a fixed rule, and write a record in the log
5. Compress the cash flow template, electronic receipts, and online banking transaction records into a folder
6. Send an email to notify of any abnormal operation and send the compressed folder to a designated email address
7. Business personnel receive the email and perform confirmation operation.


In [22]:
print(result['code'])

1. Read bank account data table and email account data table to get account information
2. Open web browser and navigate to online banking system login page
3. Input account information and click login button
4. Navigate to historical balance data page for the previous business day
5. Extract balance data for each bank account and write it into the corresponding bank account balance column in the cash flow template
6. Navigate to electronic receipt download page for the previous business day
7. Download electronic receipts and name them according to a fixed rule
8. Navigate to online banking transaction record download page for the previous business day
9. Download online banking transaction records and name them according to a fixed rule
10. Write a record in the log for each downloaded file
11. Compress the cash flow template, electronic receipts, and online banking transaction records into a folder
12. Send an email to notify of any abnormal operation and send the compressed folder 

In [23]:
# with open('patents/requirements-test4.txt', 'w') as f:
#     f.write('requirements: ' + result['requirements'])
#     f.write('\n\nfeature: ' + result['feature'])
#     f.write('\n\nsolution: ' + result['solution'])
#     f.write('\n\nselection: ' + result['selection'])

In [24]:
steps = result['code'].split('\n\n')
steps[:3]

['1. Read bank account data table and email account data table to get account information',
 '2. Open web browser and navigate to online banking system login page',
 '3. Input account information and click login button']

In [25]:
template_generate = """你是一个专业的RPA开发人员，请根据RPA库函数描述，选择合适的函数来生成一个步骤的代码。
如果实现这个任务步骤所需要的函数不存在于上下文中，则可以使用通用第三方函数库。如果存在，则必须使用已有的RPA库函数。
在生成代码时，只需要生成代码和注释，不需要别的解释。所有的注释都必须是中文。

相关函数: {context}

当前步骤：{question}

生成的代码:"""

prompt_generate = PromptTemplate(
    template=template_generate, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": prompt_generate}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [26]:
for i in range(len(steps)):
    result = qa.run(steps[i])
    print(f'>> {steps[i]} \n {result} \n')

>> 1. Read bank account data table and email account data table to get account information 
 # Read bank account data table
bank_data = read_excel('path/to/bank_data.xlsx')

# Read email account data table
email_data = read_excel('path/to/email_data.xlsx')

# Get email information
emails = receiveExchangeEmail(email_data['serverAddress'], email_data['adDomain'], email_data['username'], email_data['password'], email_data['emailAddress'], email_data['folder'], None, None, None, True, True, None, False, 10)

# Get bank card information
for index, row in bank_data.iterrows():
    bank_card_info = bankCardOCR(row['imagePath'], 'your_api_key', 'your_secret_key')
    # Process bank card information here... 

>> 2. Open web browser and navigate to online banking system login page 
 // Open web browser
let browserObject = getBrowserObject();
let tabId = getBrowserTabId();
openWebsiteInSelectedTab(browserObject, tabId, 5, "https://www.onlinebanking.com/login");

// Navigate to login page
simulat